<a href="https://colab.research.google.com/github/P2Enjoy/ia-web3.fr/blob/main/SpeedWatch_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install all requirements (run once) ~~(GOOD)

# System
import locale, os
locale.getpreferredencoding = lambda: "UTF-8"
!pip install --upgrade transformers==4.31.0 torch accelerate==0.21.0 bitsandbytes==0.41.0 tensorflow==2.13.0 onnxruntime-gpu==1.15.1
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Application for ponctuation
!pip install deepmultilingualpunctuation
!pip install scikit-learn

!pip install pytorch-crf==0.7.2 youtube_transcript_api==0.6.1 langchain sentence_transformers

# Youtube processing

In [3]:
#@title Put the link of the video to summarize:
video_link = "https://www.youtube.com/watch?v=jQL0ZeHtXFc" #@param {type:"string"}

In [4]:
#@title Extract the youtube video transcript

def YTVideoToText(video_link):
  # installing & importing libraries
  from youtube_transcript_api import YouTubeTranscriptApi

  # fetching video transcript
  video_id = video_link.split("=")[1]
  transcript = YouTubeTranscriptApi.get_transcript(video_id)

  # iterating throughout and adding all text together (recovering ponctuations)
  flat_transcript = ""
  for tu in transcript:
      flat_transcript += ' ' + tu['text']

  return flat_transcript

video_full_text = YTVideoToText(video_link)

In [5]:
#@title Restore the ponctuation

from deepmultilingualpunctuation import PunctuationModel

model = PunctuationModel()
video_restored_text = model.restore_punctuation(video_full_text)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


# Summarization

In [1]:
#@title Load the model into memory
# Load model directly
from transformers import AutoTokenizer, pipeline

model_id = "facebook/bart-large-cnn" #@param {type:"string"}
tokenizer = AutoTokenizer.from_pretrained(model_id)
summarizeryt = pipeline('summarization', model= model_id, device=0)

In [44]:
#@title Join sentences in paragraphs of less the specified token size
summarize_token_width = 1000 #@param {type:"slider", min:250, max:2048, step:250}

from langchain.text_splitter import CharacterTextSplitter
splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, separator=f'\. ', chunk_size=summarize_token_width, chunk_overlap=0
)

In [47]:
def do_sum(sentences):
  # summarize text
  summarized_text = ''
  for sentence in sentences:
      out = summarizeryt(sentence, min_length=30, do_sample=False)
      out = out[0]
      out = out['summary_text']
      summarized_text += f'.\n{out}'

  # returning summary
  return summarized_text;

video_summarized_text = do_sum(splitter.split_text(video_restored_text))

In [49]:
#@title Show the summarized text of the video
video_summarized_text

'.\nPart three of how to instruct, fine tune your large language model in a GPT language suit model. In this part of the series, we look at the Stanford alpaca prompt, the Stanford GPT prompt. The prompt asks users to come up with 20 diverse task instruction in English..\nStanford alpaca prompt takes two beautiful steps. The original human way was explain human behavior, and now it\'s a much nicer way. This is the power of gbt4 or judge gbt, I don\'t know whatever..\nJson was exactly what I, what we have in a second resident here, generate instruction- sorry, this one\\. so here we go in and what I want to show you here we start Channel 8: instruction following data, our instruction, our self instruction, generated data for fine tuning. So first we tokenize all the seed instruction. Then we go here to open AI to the utilities by openai, and open AI will create, as I just showed you here, some diverse tasks..\nThe code implementation of the Stanford alpaca generator instruction, a pytho